In [2]:
from bs4 import BeautifulSoup
from bs4.element import Comment
from urlparse import urlparse
import pandas as pd
import openpyxl as op
import os
import urllib2, re
import ssl
import time


os.chdir('/Users/krothaps/Desktop/hackathon/dataset/tim_first_level/')

In [3]:
def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    if isinstance(element, Comment):
        return False
    return True

def fetch_text_from_url(url):
    html = urllib2.urlopen(url).read()
    soup = BeautifulSoup(html, "html.parser")
    data = soup.findAll(text=True)
 
    result = filter(visible, data)

    page = ' '.join(str(e.encode('utf-8')) for e in result)
    page = page.replace('\n', ' ').replace('\t', ' ').replace('\r', '')
    return page

In [4]:
def fetch_recursive_urls(univ_url, univ_name):
    university_url = univ_url
#     univ_name = re.sub(r'\W+', '', univ_name)
    university_name = univ_name.lower().strip().replace(" ", "_")
    output_file_name = university_name+'.txt'
    first_level_urls = []
    second_level_urls = []
    parsed_urls = []
    random_flag = 0
    output_file = open(output_file_name, 'w')

    ssl._create_default_https_context = ssl._create_unverified_context

    html_page = urllib2.urlopen(university_url).read()
#     html_page2 = urllib2.urlopen(university_url)
    soup = BeautifulSoup(html_page, "lxml")
#     soup2 = BeautifulSoup(html_page2, "lxml")

    
    # fetch text from the main page
    data = soup.findAll(text=True)
    result = filter(visible, data)
    page = ' '.join(str(e.encode('utf-8')) for e in result)
    page = page.replace('\n', ' ').replace('\t', ' ').replace('\r', '')
    parsed_urls.append(university_url)
    
    output_file.write(university_name+'\t'+university_url+'\t'+university_url+'\t'+page+'\n')

    start_time = time.time()

    tag_data1 = soup.findAll('a', attrs={'href': re.compile("^http://")})
    tag_data2 = soup.findAll('a', attrs={'href': re.compile("^https://")})
    links1 = [link.get('href').encode('utf-8').strip() for link in tag_data1]
    links2 = [link.get('href').encode('utf-8').strip() for link in tag_data2]
    links = list(set(links1+links2))
#     print links
 
    for link in links:
        try:
            if link not in parsed_urls and '.edu' in link:
                url_text = fetch_text_from_url(link)
                output_file.write(university_name+'\t'+university_url+'\t'+link+'\t'+ url_text+'\n')
            else:
                url_text = 'not-parsed'
                output_file.write(university_name+'\t'+university_url+'\t'+link+'\t'+ url_text+'\n')
            parsed_urls.append(link)    
        except:
            random_flag += 1
            
    execution_time = time.time() - start_time


    output_file.close()
    print univst_name+','+univ_url+','+str(len(links))+','+str(random_flag)+','+str(int(execution_time))

mit_demo,http://www.mit.edu/,16,0,5


In [6]:
with open('/Users/krothaps/Desktop/hackathon/dataset/tim_list_v1.csv') as f:
    content = f.readlines()

# you may also want to remove whitespace characters like `\n` at the end of each line
# content = [x.strip() for x in content][0]
# print len(content)
# lines = content.split('\n')

print len(content)
# print len(lines)

for line in content:

    if len(line.strip().split(',')) == 4:
        univ_name = line.strip().split(',')[0]
        university_name = str(univ_name)
        output_file_name = str(university_name)+'.txt'

        univ_url = line.strip().split(',')[2].strip()
        if 'url_exists' == line.strip().split(',')[3] and not os.path.isfile(output_file_name):
            try:
                fetch_recursive_urls(univ_url, univ_name)
            except:
                print 'hi'
    
f.close()

1000


In [7]:
def fetch_univ_url(line):
    line_list = line.strip().split(',')
    if line_list[2] != '#N/A' and '.edu' in line_list[2]:
        return line_list[2]
    elif 'http' in line_list[4]:
        return line_list[4]
    elif 'www' in line_list[4]:
        return 'http://'+line_list[4]
    elif 'edu' in line_list[4]:
        return 'http://www.'+line_list[4]
    else:
        return 'none'

In [8]:
with open('/Users/krothaps/Desktop/hackathon/dataset/tim_list_v2.csv','rU') as f:
    content = f.readlines()

# you may also want to remove whitespace characters like `\n` at the end of each line
# content = [x.strip() for x in content][0]
# print len(content)
# lines = content.split('\n')

# print len(content)
# print content
# print len(lines)

for line in content:

    if len(line.strip().split(',')) == 5:
        
        univ_name = line.strip().split(',')[0]
        output_file_name = str(univ_name)+'.txt'

        univ_url = fetch_univ_url(line)
        if not os.path.isfile(output_file_name) and 'edu' in univ_url and univ_url != 'none':
            try:
                fetch_recursive_urls(univ_url, univ_name)
            except:
                print 'hi'
    
f.close()

No handlers could be found for logger "bs4.dammit"


106412,http://www.uapb.edu,127,13,827
175421,http://www.belhaven.edu,67,2,36
215099,http://www.philau.edu,0,0,0
hi
174075,http://www.umcrookston.edu/,26,0,12
186432,http://www.saintpeters.edu,114,0,298
156286,http://www.bellarmine.edu,21,2,8
hi
133711,http://www.flagler.edu,12,1,1
hi
159416,http://www.lsus.edu/,11,0,1
195234,http://www.strose.edu,111,2,273
195173,http://www.sfc.edu,31,1,43
hi
482699,http://www.sgsc.edu,26,0,7
238980,http://www.lakeland.edu,14,0,1
173045,http://www.augsburg.edu,16,1,1
204705,http://www.newark.osu.edu,57,3,37
184694,http://www.fdu.edu,54,1,10
hi
162283,http://www.coppin.edu,39,0,18
128391,http://www.western.edu,64,1,4
204936,http://www.otterbein.edu,34,2,20
139010,http://www.bainbridge.edu,2,0,1
213987,http://www.mercyhurst.edu,85,1,421
219347,http://www.sdsmt.edu,24,0,10
hi
203526,http://www.geauga.kent.edu,25,0,12
154235,http://www.sau.edu,9,2,3
207564,http://www.osuit.edu/,114,0,81
hi
104586,http://www.erau.edu,0,0,0
203474,http://www.trumbull.kent.ed